In [ ]:
import pandas as pd
import scipy.optimize as opt
import regularized_optimization as reg_opt
import Utilityfunctions as utils
import jax.numpy as jnp
import numpy as np
import jax as jax
import matplotlib.pyplot as plt
import simulations as simul
jax.config.update("jax_enable_x64", True)
plt.rcParams['text.usetex'] = True
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
params = pd.read_csv("../results/luad/luad_diag_unpenal_002.csv", index_col=0)
th_learned = params.iloc[:,:-1].to_numpy()
lam1 = np.exp(params.iloc[0,-1])
lam2 = np.exp(params.iloc[1, -1])
events = params.columns[:-1]
n_sim = 100000
n = th_learned.shape[0]
rng = np.random.default_rng(42)

In [ ]:
utils.plot_theta(params.iloc[:,:-1], 1)

In [ ]:
probs_prim, probs_met = simul.p_full_orders(th_learned, n_sim, lam1, lam2, rng)
cum_probs_prim = np.cumsum(probs_prim, axis=0)
cum_probs_met = np.cumsum(probs_met, axis=0)

In [ ]:
with PdfPages('paad_orders.pdf') as pdf:
    for i in range(n):
        fig,ax = plt.subplots(ncols=1, nrows=2, figsize=(11.69,8.27))
        
        ax[0].set_title(events[i]+": Order of accumulation")
        ax[0].bar(np.arange(1,21), probs_met[:20,i], color="#D55E00", label="MT")
        ax[0].bar(np.arange(1,21), probs_prim[:20,i], label="PT", color="#56B4E9", alpha=0.8)
        ax[0].set_xticks(np.arange(0,21))
        ax[0].legend()

        ax[1].set_title(events[i]+": Cumulative probability")
        ax[1].plot(np.arange(1,21), cum_probs_prim[:20,i], label="PT", linewidth="4", color="#56B4E9", alpha=0.8)
        ax[1].plot(np.arange(1,21), cum_probs_met[:20,i], color="#D55E00", label="MT", linewidth="4")
        ax[1].set_xticks(np.arange(0,21))
        ax[1].legend()

        pdf.savefig()
        plt.close()

Calculate $P(t_{mut} < t_{seed}| x[mut] = 1)$ i.e. the probability that a mutation happend prior to the seeding for both primary tumors and metastases:

In [ ]:
mut_ps, mut_total_prim, mut_total_met = simul.p_any_mut_pre_seed(th_learned, n_sim, lam1, lam2, rng)

In [ ]:
psp = pd.DataFrame(np.column_stack((mut_ps/mut_total_prim, mut_ps/mut_total_met)), 
                  index=events[:-1], columns=["Prim", "Met"])
psp.sort_values(by=["Prim"], inplace=True)
fig, ax = plt.subplots(figsize=(8.27, 11.69))
plt.style.use("default")
ax.set_xlim(0,1)
ax.axvline(0.5, color = "#D55E00", linestyle="dashed")
psp["Prim"].plot(kind="barh", color="#56B4E9", ax=ax, position=0, width=0.4)
psp["Met"].plot(kind="barh", ax=ax, color="#D55E00", position=1, width=0.4)
ax.legend()

Calculate $P(t_{mut} < t_{seed} | x[mut] = z[mut] = 1)$ ie. the probability that a **shared** mutation between the primary tumor and the metastasis happened prior to the seeding

In [ ]:
mut_ps, mut_total = simul.p_shared_mut_pre_seed(th_learned, n_sim, lam1, lam2, rng)
psp = pd.DataFrame(mut_ps/mut_total, index=events[:-1], columns=["Shared"])
psp.sort_values(by=["Shared"], inplace=True)
fig, ax = plt.subplots(figsize=(8.27, 11.69))
plt.style.use("default")
ax.set_xlim(0,1)
ax.axvline(0.5, color = "#D55E00", linestyle="dashed")
psp["Prim"].plot(kind="barh", color="#56B4E9", ax=ax)
ax.legend()

In [ ]:
res, iwas1, iwas2 = simul.simulate_dat(th_learned, 100000, lam1, lam2, rng)
res[:, -1].sum()/100000